# Building a prediction API using a pre-trained MXNet model

In [56]:
'''
Sample handler for inference using Keras on AWS Lambda

@author: Sunil Mallya
'''

import os

# Check if Lambda Function
if os.environ.get('LAMBDA_TASK_ROOT') is None:
    print "just exit, we are not in a lambda function",
    import sys; sys.exit(0)

import json
import numpy as np
import tempfile
import time
import urllib2

from keras.models import Sequential
from keras.layers import Dense
from PIL import Image

# For ImageNet models 
from keras_squeezenet import SqueezeNet #using this lib to test SqueezeNet
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
model = SqueezeNet()

def download_image(url):

    req = urllib2.urlopen(url)
    img_file = tempfile.NamedTemporaryFile(delete=True)
    img_file.write(req.read())
    img_file.flush()
    img = Image.open(img_file.name)
    img = img.resize((227, 227), Image.ANTIALIAS)
    x = np.asarray(img)
    x = np.expand_dims(x, axis=0)
    return x

def lambda_handler(event, context):
    
    url = ''
    try:
        # API Gateway GET method
        if event['httpMethod'] == 'GET':
            url = event['queryStringParameters']['url']
        # API Gateway POST method
        elif event['httpMethod'] == 'POST':
            data = json.loads(event['body'])
            url = data['url']
    except KeyError:
        # direct invocation
        url = event['url']
    
    handler_start_time = time.time()
    start_time = time.time()
    x = download_image(url)
    #requires scipy lib, can't use it since that puts us over 50MB zip limit for Lambda
    #x = preprocess_input(x) 
    preds = model.predict(x)
    end_time = time.time()
    # [[(u'n02088364', u'beagle', 0.94316888), (u'n04254680', u'soccer_ball', 0.017797621),...]]
    pred_lst = decode_predictions(preds)
    outputs = []
    for _id, lbl, prob in pred_lst[0]:
        ele = {}
        ele["label"] = lbl
        ele["prob"] = str(prob)
        outputs.append(ele)
        
    print('Predicted:', outputs)
    h_time = end_time - handler_start_time
    p_time = end_time - start_time
    print("handler:", h_time ,"pred time:", p_time)
    #return "%s,%s" % (h_time, p_time) 
    out = {
            "headers": {
                "content-type": "application/json",
                "Access-Control-Allow-Origin": "*"
                },
            "body": '{"labels":"%s", "handler_time": "%s", "prediction_time": "%s"}' % (json.dumps(outputs), h_time, p_time),  
            "statusCode": 200
          }
    return out

just exit, we are not in a lambda function

SystemExit: 0

In [57]:
# Package code and upload 

# refer for more info: http://ipython.readthedocs.io/en/stable/interactive/reference.html?highlight=input%20caching

content = _ih[-2] # Get the input from the previous cell execution 
fname = "lambda_function.py"

with open(fname, 'w') as f:
    f.write(content)

In [58]:
# NOTE: Pre-requisities (AWS CLI) and appropriate config credentials 
code_zip_name = 'keras_lambda_code.zip'

# Create a zip file with all of Keras-TF dependencies (-Fsr : Only sync updated files)
!zip -9r -qFSr $code_zip_name * -x *.zip 

In [ ]:
# UPDATE CODE - Easy to debug

func_name = "Keras-Lambda-SqueezeNet-LambdaFunction-1RP44Y0PT0O3G"
!aws lambda update-function-code --function-name $func_name --zip-file fileb://$code_zip_name  --region us-west-2

# Swagger File

In [4]:
account_id = 'MY_ACC_ID'
region = 'us-west-2'

!sed -e 's/<<region>>/$region/g' swagger.yaml.template > swagger.yaml
!sed -i -e 's/<<account-id>>/$account_id/g' swagger.yaml

# Upload Code and YAML files to S3

In [ ]:
bucket_loc = "s3://MY_BUCKET/keras-samtest/" # **NOTE** Make sure bucket is in the same region as region above
!aws s3 cp $code_zip_name $bucket_loc
!aws s3 cp swagger.yaml $bucket_loc

# Template File

In [6]:
definition_url = bucket_loc + 'swagger.yaml' # swagger file location in s3
code_uri = bucket_loc + code_zip_name  # code location in s3
definition_url = definition_url.replace(':', '\:').replace('/', '\/')
code_uri = code_uri.replace(':', '\:').replace('/', '\/')

!sed -e 's/<<def-uri>>/$definition_url/g' template.yaml.template > template.yaml
!sed -i -e 's/<<code-uri>>/$code_uri/g' template.yaml

# Deply using SAM 

In [7]:
!aws cloudformation package \
 --template-file template.yaml \
 --output-template-file template-out.yaml \
 --s3-bucket $bucket_loc


Successfully packaged artifacts and wrote output template to file template-out.yaml.
Execute the following command to deploy the packaged template
aws cloudformation deploy --template-file /home/ubuntu/workspace/keras-tf-lambda-working/template-out.yaml --stack-name <YOUR STACK NAME>


In [8]:
stack_name = "Keras-Lambda-SqueezeNet"

!aws cloudformation deploy \
--template-file template-out.yaml \
--stack-name $stack_name \
--capabilities CAPABILITY_IAM \
--region $region

Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - Keras-Lambda-SqueezeNet


In [ ]:
api_endpoint = !aws cloudformation describe-stacks --stack-name $stack_name --region $region | python -c 'import json,sys;obj=json.load(sys.stdin);print obj["Stacks"][0]["Outputs"][0]["OutputValue"];'
print api_endpoint

In [61]:
import json

img_url = 'https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg'
url = api_endpoint[0]+ "/predict?url=" + img_url

# Lets curl and test the endpoint
!curl $url

{"labels":"[{"prob": "0.943169", "label": "beagle"}, {"prob": "0.0177976", "label": "soccer_ball"}, {"prob": "0.0175641", "label": "Walker_hound"}, {"prob": "0.00897344", "label": "Labrador_retriever"}, {"prob": "0.00510975", "label": "basset"}]", "handler_time": "0.320521116257", "prediction_time": "0.320521116257"}